In [14]:
# cell to add at index 0
import pandas as pd
from typing import Optional

CSV_PATH = "assets/worldcities.csv"

def load_worldcities(path: str = CSV_PATH) -> pd.DataFrame:
    """
    Load worldcities CSV and ensure a numeric 'population' column exists.
    """
    df = pd.read_csv(path)
    # try to find a population-like column
    pop_candidates = [c for c in df.columns if c.lower() in ("population", "pop", "pop_est", "pop2019")]
    if not pop_candidates:
        raise ValueError(f"No population-like column found in {path}. Columns: {list(df.columns)}")
    pop_col = pop_candidates[0]
    df["population"] = pd.to_numeric(df[pop_col], errors="coerce")
    return df

def filter_by_population(
    df: pd.DataFrame,
    min_pop: Optional[float] = None,
    max_pop: Optional[float] = None,
    top_n: Optional[int] = None,
    dropna: bool = True,
    sort_desc: bool = True
) -> pd.DataFrame:
    """
    Return rows filtered by population.
    - min_pop / max_pop: inclusive thresholds
    - top_n: return only the top N rows by population (applies after filtering)
    - dropna: drop rows with missing population before applying thresholds
    - sort_desc: sort by population descending if True
    """
    cond = pd.Series(True, index=df.index)
    if dropna:
        cond &= df["population"].notna()
    if min_pop is not None:
        cond &= df["population"] >= min_pop
    if max_pop is not None:
        cond &= df["population"] <= max_pop
    out = df[cond].copy()
    if sort_desc:
        out = out.sort_values("population", ascending=False)
    if top_n is not None:
        out = out.head(top_n)
    return out

# Example usage
if __name__ == "__main__":
    df = load_worldcities()
    # cities with at least 1,000,000 people
    million_plus = filter_by_population(df, min_pop=20_000_000)
    # print number of entries in the filtered "database"
    print(million_plus.shape[0])

13


In [1]:
# Save as check_gpu.py and run it
import torch

if not torch.cuda.is_available():
    print("ERROR: CUDA is not available. You are running on CPU.")
else:
    print(f"Found {torch.cuda.device_count()} GPUs:")
    for i in range(torch.cuda.device_count()):
        print(f"   [ID: {i}] {torch.cuda.get_device_name(i)}")

Found 3 GPUs:
   [ID: 0] NVIDIA RTX A4000
   [ID: 1] NVIDIA RTX A4000
   [ID: 2] NVIDIA T400


In [1]:
import sqlite3
import os

DB_PATH = "data/pipeline.db"

def reset_database():
    if not os.path.exists(DB_PATH):
        print("❌ Database not found. Nothing to reset.")
        return

    # Connect with isolation_level=None allows us to manage transactions manually
    # or run commands like VACUUM that require no active transaction.
    conn = sqlite3.connect(DB_PATH) 
    cursor = conn.cursor()

    try:
        print("--- 🔄 Resetting Database ---")
        
        # 1. Wipe data (Inside a transaction for safety)
        print("1. Deleting all detections...")
        cursor.execute("DELETE FROM mapillary_detections")
        
        print("2. Deleting all images...")
        cursor.execute("DELETE FROM mapillary_images")

        print("3. Resetting all cities to 'pending'...")
        cursor.execute("""
            UPDATE cities 
            SET download_status = 'pending', 
                analysis_status = 'pending', 
                downloaded_at = NULL, 
                analyzed_at = NULL
        """)

        # Commit the deletions immediately so the transaction closes
        conn.commit()

        # 2. Optimize DB file size (Must be outside of a transaction)
        print("4. Vacuuming database (reclaiming space)...")
        
        # Ensure we are in autocommit mode for VACUUM
        conn.isolation_level = None 
        cursor.execute("VACUUM")
        
        print("✅ Database reset complete. All cities are ready to be rescanned.")

    except Exception as e:
        print(f"❌ Error during reset: {e}")
        # Only rollback if we haven't committed yet
        if conn.in_transaction:
            conn.rollback()
    finally:
        conn.close()

if __name__ == "__main__":
    reset_database()

--- 🔄 Resetting Database ---
1. Deleting all detections...
2. Deleting all images...
3. Resetting all cities to 'pending'...
4. Vacuuming database (reclaiming space)...
✅ Database reset complete. All cities are ready to be rescanned.


# Efficiency analysis of batch processing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- Configuration ---
LOG_FILE = "performance_log.csv"

# Set plot style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

def analyze_performance():
    # 1. Load Data
    try:
        df = pd.read_csv(LOG_FILE)
    except FileNotFoundError:
        print(f"❌ Error: '{LOG_FILE}' not found. Run the detection pipeline first to generate logs.")
        return

    # Convert Timestamp to datetime
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    
    # Calculate 'Elapsed Time' for plotting trends relative to start
    df = df.sort_values(by='Timestamp')
    df['Elapsed_Seconds'] = (df['Timestamp'] - df['Timestamp'].min()).dt.total_seconds()

    # 2. Summary Statistics by Batch Size
    print("--- 📊 Performance Summary by Batch Size ---")
    summary = df.groupby('Batch_Size')['Current_FPS'].agg(
        Mean_FPS='mean',
        Median_FPS='median',
        Min_FPS='min',
        Max_FPS='max',
        Std_Dev='std',
        Count='count'
    ).round(2)
    
    # Calculate Total Throughput (Images / Total Time) per Batch configuration
    # (Approximate based on sum of batch durations)
    throughput_df = df.groupby('Batch_Size').agg(
        Total_Images=('Images', 'sum'),
        Total_Duration=('Batch_Time_Sec', 'sum')
    )
    throughput_df['Real_Throughput_FPS'] = (throughput_df['Total_Images'] / throughput_df['Total_Duration']).round(2)
    
    # Combine stats
    final_summary = pd.concat([summary, throughput_df['Real_Throughput_FPS']], axis=1)
    display(final_summary)

    # 3. Visualizations
    
    # A. Box Plot (Stability & Distribution)
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Batch_Size', y='Current_FPS', data=df, palette="viridis")
    plt.title('FPS Stability Distribution by Batch Size', fontsize=16)
    plt.ylabel('Frames Per Second (FPS)', fontsize=12)
    plt.xlabel('Batch Size', fontsize=12)
    plt.show()

    # B. FPS Trend Over Time (Warmup & Throttle check)
    plt.figure(figsize=(14, 6))
    sns.lineplot(x='Elapsed_Seconds', y='Current_FPS', hue='Batch_Size', data=df, palette="tab10", alpha=0.7)
    plt.title('FPS Trend Over Time (Check for Throttling)', fontsize=16)
    plt.ylabel('FPS', fontsize=12)
    plt.xlabel('Seconds since start', fontsize=12)
    plt.legend(title='Batch Size')
    plt.show()

    # C. Batch Processing Time vs Batch Size
    # Helps visualize latency (how long you wait for one result)
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Batch_Size', y='Batch_Time_Sec', data=df, errorbar='sd', palette="magma")
    plt.title('Average Processing Time per Batch (Latency)', fontsize=16)
    plt.ylabel('Seconds per Batch', fontsize=12)
    plt.xlabel('Batch Size', fontsize=12)
    for container in plt.gca().containers:
        plt.gca().bar_label(container, fmt='%.2fs')
    plt.show()

    # 4. Recommendation Engine
    best_batch = final_summary['Real_Throughput_FPS'].idxmax()
    best_fps = final_summary['Real_Throughput_FPS'].max()
    
    print(f"\n🏆 **Winner:** Batch Size {best_batch} with {best_fps} FPS.")
    
    # Check for instability
    unstable_batches = final_summary[final_summary['Std_Dev'] > 2.0].index.tolist()
    if unstable_batches:
        print(f"⚠️ Warning: Batch sizes {unstable_batches} show high variance (instability).")

if __name__ == "__main__":
    analyze_performance()

ParserError: Error tokenizing data. C error: Expected 7 fields in line 74, saw 8
